In [1]:
from utils import create_facts_and_examples
import numpy as np
import pandas as pd
import random, os
np.random.seed(0) #Code B

In [2]:
N = 30
A =  np.random.randint(2, size=N)
B =  np.random.randint(2, size=N)
C =  np.random.randint(2, size=N)

customer = ['c'+str(i+16000) for i in range(90)]

Fraud = A & B # & C
df = pd.DataFrame({'orig':np.random.choice(customer, size=N), 'destination':np.random.choice(customer, size=N),\
                   'A':A, 'B':B, "C":C, "Fraud":Fraud})
df.drop(df[(A==0)&(B==0)&(C==0)].index, inplace=True)

df = pd.DataFrame({'orig':np.random.choice(customer, size=N), 'destination':np.random.choice(customer, size=N),\
                   'A':A, 'B':B, "Fraud":Fraud})
df.drop(df[(A==0)&(B==0)].index, inplace=True)

In [3]:
df

,orig,destination,A,B,Fraud
0,c16058,c16066,0,1,0
1,c16065,c16052,1,0,0
2,c16036,c16067,1,1,1
4,c16086,c16014,1,1,1
5,c16043,c16004,1,1,1
6,c16011,c16067,1,0,0
7,c16002,c16011,1,1,1
8,c16051,c16086,1,1,1
9,c16080,c16077,1,0,0
10,c16032,c16075,1,0,0


In [4]:
df.drop(df[df['orig']==df['destination']].index, inplace=True)



In [5]:
fraudsters = df[df['Fraud']==True]

In [6]:
fraudsters[['orig','destination']]

,orig,destination
2,c16036,c16067
4,c16086,c16014
5,c16043,c16004
7,c16002,c16011
8,c16051,c16086
13,c16038,c16024
19,c16077,c16061
21,c16024,c16033
22,c16002,c16032
26,c16072,c16013


### Erman's Idea

Target is Fraud  (1) iff A and B and C and D (are 1)

Main idea: We need to learn a rule: 

Fraud_trans(U1, U2):- Transaction(X), Orig(X, U1), Dest(X, U2),  Fraud(X).

Fraud_chain(U1, U2):- Fraud_trans(U1, Y), Fraud_chain(Y, U2),



In [7]:
index_from_merge = pd.merge(df.reset_index(), fraudsters[['destination']].drop_duplicates(), how='inner', left_on='orig', right_on='destination')

In [8]:
index_from_merge

,index,orig,destination_x,A,B,Fraud,destination_y
0,4,c16086,c16014,1,1,1,c16086
1,6,c16011,c16067,1,0,0,c16011
2,10,c16032,c16075,1,0,0,c16032
3,21,c16024,c16033,1,1,1,c16024
4,24,c16013,c16085,0,1,0,c16013


In [9]:
df['Fraud_chain->orig->destination']=0
df.loc[index_from_merge['index'],'Fraud']=1
df.loc[index_from_merge['index'],'Fraud_chain->orig->destination']=1
df['From->orig']=1
df['To->destination']=1

In [10]:
df.loc[index_from_merge['index'],:]

,orig,destination,A,B,Fraud,Fraud_chain->orig->destination,From->orig,To->destination
4,c16086,c16014,1,1,1,1,1,1
6,c16011,c16067,1,0,1,1,1,1
10,c16032,c16075,1,0,1,1,1,1
21,c16024,c16033,1,1,1,1,1,1
24,c16013,c16085,0,1,1,1,1,1


In [11]:
df

,orig,destination,A,B,Fraud,Fraud_chain->orig->destination,From->orig,To->destination
0,c16058,c16066,0,1,0,0,1,1
1,c16065,c16052,1,0,0,0,1,1
2,c16036,c16067,1,1,1,0,1,1
4,c16086,c16014,1,1,1,1,1,1
5,c16043,c16004,1,1,1,0,1,1
6,c16011,c16067,1,0,1,1,1,1
7,c16002,c16011,1,1,1,0,1,1
8,c16051,c16086,1,1,1,0,1,1
9,c16080,c16077,1,0,0,0,1,1
10,c16032,c16075,1,0,1,1,1,1


### creating fact, positive and negative examples

In [12]:
#drop first letter and convert to int
df['orig'] = df['orig'].str[1:].astype('int')
df['destination'] = df['destination'].str[1:].astype('int')

In [13]:
create_facts_and_examples(df_= df, target='Fraud', 
                          predicates=['A', 'B', 'From->orig', 'To->destination'], output_dir="examples/chain-fraud-erman")

[ 2  4  5  6  7  8 10 13 19 21 22 24 26]
[ 0  1  9 11 14 15 16 17 23 25 27 28 29]
A
[ 1  2  4  5  6  7  8  9 10 13 19 21 22 25 26 27 28]
B
[ 0  2  4  5  7  8 11 13 14 15 16 17 19 21 22 23 24 26 29]
From->orig
From->orig 0 16058 flag= True
From->orig 1 16065 flag= True
From->orig 2 16036 flag= True
From->orig 4 16086 flag= True
From->orig 5 16043 flag= True
From->orig 6 16011 flag= True
From->orig 7 16002 flag= True
From->orig 8 16051 flag= True
From->orig 9 16080 flag= True
From->orig 10 16032 flag= True
From->orig 11 16054 flag= True
From->orig 13 16038 flag= True
From->orig 14 16019 flag= True
From->orig 15 16046 flag= True
From->orig 16 16042 flag= True
From->orig 17 16056 flag= True
From->orig 19 16077 flag= True
From->orig 21 16024 flag= True
From->orig 22 16002 flag= True
From->orig 23 16003 flag= True
From->orig 24 16013 flag= True
From->orig 25 16040 flag= True
From->orig 26 16072 flag= True
From->orig 27 16019 flag= True
From->orig 28 16072 flag= True
From->orig 29 16026 flag=

In [14]:
create_facts_and_examples(df_= df, target='Fraud_chain->orig->destination', 
                          predicates=['Fraud', 'From->orig', 'To->destination'], output_dir="examples/transitivity-fraud-rule")

Fraud_chain->orig->destination 16086 16014 flag= True
Fraud_chain->orig->destination 16011 16067 flag= True
Fraud_chain->orig->destination 16032 16075 flag= True
Fraud_chain->orig->destination 16024 16033 flag= True
Fraud_chain->orig->destination 16013 16085 flag= True
Fraud_chain->orig->destination 16058 16066 flag= False
Fraud_chain->orig->destination 16065 16052 flag= False
Fraud_chain->orig->destination 16036 16067 flag= False
Fraud_chain->orig->destination 16043 16004 flag= False
Fraud_chain->orig->destination 16002 16011 flag= False
Fraud_chain->orig->destination 16051 16086 flag= False
Fraud_chain->orig->destination 16080 16077 flag= False
Fraud_chain->orig->destination 16054 16056 flag= False
Fraud_chain->orig->destination 16038 16024 flag= False
Fraud_chain->orig->destination 16019 16029 flag= False
Fraud_chain->orig->destination 16046 16021 flag= False
Fraud_chain->orig->destination 16042 16025 flag= False
Fraud_chain->orig->destination 16056 16080 flag= False
Fraud_chain->or

## Create simple transitivity problem

In [15]:
df

,orig,destination,A,B,Fraud,Fraud_chain->orig->destination,From->orig,To->destination
0,16058,16066,0,1,0,0,1,1
1,16065,16052,1,0,0,0,1,1
2,16036,16067,1,1,1,0,1,1
4,16086,16014,1,1,1,1,1,1
5,16043,16004,1,1,1,0,1,1
6,16011,16067,1,0,1,1,1,1
7,16002,16011,1,1,1,0,1,1
8,16051,16086,1,1,1,0,1,1
9,16080,16077,1,0,0,0,1,1
10,16032,16075,1,0,1,1,1,1


In [16]:
df['Fraud->orig->destination']=df['Fraud']
df['Transfer->orig->destination']=1
create_facts_and_examples(df_= df, target='Fraud_chain->orig->destination', 
                          predicates=['Fraud->orig->destination', 'Transfer->orig->destination'], output_dir="examples/simple-transitivity-fraud-rule")

Fraud_chain->orig->destination 16086 16014 flag= True
Fraud_chain->orig->destination 16011 16067 flag= True
Fraud_chain->orig->destination 16032 16075 flag= True
Fraud_chain->orig->destination 16024 16033 flag= True
Fraud_chain->orig->destination 16013 16085 flag= True
Fraud_chain->orig->destination 16058 16066 flag= False
Fraud_chain->orig->destination 16065 16052 flag= False
Fraud_chain->orig->destination 16036 16067 flag= False
Fraud_chain->orig->destination 16043 16004 flag= False
Fraud_chain->orig->destination 16002 16011 flag= False
Fraud_chain->orig->destination 16051 16086 flag= False
Fraud_chain->orig->destination 16080 16077 flag= False
Fraud_chain->orig->destination 16054 16056 flag= False
Fraud_chain->orig->destination 16038 16024 flag= False
Fraud_chain->orig->destination 16019 16029 flag= False
Fraud_chain->orig->destination 16046 16021 flag= False
Fraud_chain->orig->destination 16042 16025 flag= False
Fraud_chain->orig->destination 16056 16080 flag= False
Fraud_chain->or

In [17]:
create_facts_and_examples(df_= df, target='Fraud_chain->orig->destination', 
                          predicates=['Fraud->orig->destination'], output_dir="examples/fraud-and-fraud-chain")

Fraud_chain->orig->destination 16086 16014 flag= True
Fraud_chain->orig->destination 16011 16067 flag= True
Fraud_chain->orig->destination 16032 16075 flag= True
Fraud_chain->orig->destination 16024 16033 flag= True
Fraud_chain->orig->destination 16013 16085 flag= True
Fraud_chain->orig->destination 16036 16067 flag= False
Fraud_chain->orig->destination 16043 16004 flag= False
Fraud_chain->orig->destination 16002 16011 flag= False
Fraud_chain->orig->destination 16051 16086 flag= False
Fraud_chain->orig->destination 16038 16024 flag= False
Fraud_chain->orig->destination 16077 16061 flag= False
Fraud_chain->orig->destination 16002 16032 flag= False
Fraud_chain->orig->destination 16072 16013 flag= False
Fraud->orig->destination
Fraud->orig->destination 16036 16067 flag= True
Fraud->orig->destination 16086 16014 flag= True
Fraud->orig->destination 16043 16004 flag= True
Fraud->orig->destination 16011 16067 flag= True
Fraud->orig->destination 16002 16011 flag= True
Fraud->orig->destination 